In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-training-label/cleaned_training_label.csv
/kaggle/input/cleaned-submission/submission2_cleaned.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/training_label.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv


Selecting first 1000 rows of gee_features for analysis

In [2]:
sample = pd.read_csv('/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv',nrows = 1000)

/tmp/ipykernel_33/4121637694.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sample = pd.read_csv('/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv',nrows = 1000)


In [3]:
sample.head()

,new_ind,index,ADM1DHS,ADM1FIPS,ADM1FIPSNA,ADM1NAME,ADM1SALBCO,ADM1SALBNA,ADM1SALNA,ALT_DEM,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,AL_2008_00000001,0,9999,NaN,NaN,NaN,NaN,NaN,NaN,45.0,...,226.56590,0.025174,52.417587,0.043573,216.05801,0.014725,47.410877,AL,1,2008
1,AL_2008_00000002,40,9999,NaN,NaN,NaN,NaN,NaN,NaN,65.0,...,225.83365,0.029437,56.164467,0.039097,200.14983,0.012322,38.932705,AL,2,2008
2,AL_2008_00000003,80,9999,NaN,NaN,NaN,NaN,NaN,NaN,261.0,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3,2008
3,AL_2008_00000004,120,9999,NaN,NaN,NaN,NaN,NaN,NaN,29.0,...,233.25563,0.025747,53.518450,0.043162,216.46187,0.014495,46.634487,AL,4,2008
4,AL_2008_00000005,160,9999,NaN,NaN,NaN,NaN,NaN,NaN,130.0,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5,2008


In [4]:
# Non Predictive features that can be deleted
Non_predictive_features = [
    "new_ind",
    "index",
    "ADM1DHS",
    "ADM1FIPS",
    "ADM1FIPSNA",
    "ADM1NAME",
    "ADM1SALBCO",
    "ADM1SALBNA",
    "ADM1SALNA",
    "ALT_DEM",
    "ALT_GPS",
    "CCFIPS",
    "DATUM",
    "DHSCC",
    "DHSCLUST",
    "DHSREGCO",
    "DHSREGNA",
    "DHSYEAR",
    "F21",
    "F22",
    "F23",
    "LATNUM",
    "LONGNUM",
    "SOURCE",
    "URBAN_RURA",
    "ZONECO",
    "ZONENA",
]

In [5]:
# Droping these columns from the sample
df1 = sample.drop(columns=Non_predictive_features)

Missing values

In [6]:
# Calculate the missing value portion for the 1000 rows
missing_portion_1 = df1.isna().mean()

# Sort the missing portion in descending order
missing_portion_1 = missing_portion_1.sort_values(ascending=False)
    
# Find the columns to drop which are greater than 60%
drop_columns_1 = missing_portion_1[missing_portion_1 >= 0.6].index

In [7]:
# length of those columns
len(drop_columns_1 )

583

In [8]:
# Droping these columns from the data frame
df2 = df1.drop(columns = drop_columns_1)

In [9]:
df2.head()

,DHSID,ET_water_median@CAS/IGSNRR/PML/V2&timestamped,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,qc_median@CAS/IGSNRR/PML/V2&timestamped,ET_water_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_mean@CAS/IGSNRR/PML/V2&timestamped,Ei_mean@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,AL200800000001,-0.802911,0.697994,0.051240,0.720611,2.867798,0.0,-0.385094,1.004067,0.063028,...,226.56590,0.025174,52.417587,0.043573,216.05801,0.014725,47.410877,AL,1,2008
1,AL200800000002,-0.802911,0.546368,0.072052,0.641110,2.831818,0.0,-0.385094,0.949528,0.080869,...,225.83365,0.029437,56.164467,0.039097,200.14983,0.012322,38.932705,AL,2,2008
2,AL200800000003,-0.802911,0.474569,0.040593,0.809169,2.093236,0.0,-0.385094,0.784751,0.055926,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3,2008
3,AL200800000004,-0.802911,0.793127,0.056218,0.680800,3.167071,0.0,-0.385094,1.091989,0.066411,...,233.25563,0.025747,53.518450,0.043162,216.46187,0.014495,46.634487,AL,4,2008
4,AL200800000005,-0.802911,0.689465,0.072462,0.696633,2.979739,0.0,-0.385094,0.930896,0.080958,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5,2008


In [10]:
# Making a copy of the dataframe
df3 =df2.copy()

In [11]:
# Loading the preprocessed training label
train_label = pd.read_csv('/kaggle/input/preprocessed-training-label/cleaned_training_label.csv')

In [12]:
train_label.head()

,Unnamed: 0,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,0,AL200800000001,24.120000,25.280000,50.000000,9.680000,100.000000,20.000000
1,1,AL200800000002,23.040000,21.980000,7.690000,8.330000,66.670000,0.000000
2,2,AL200800000003,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
3,3,AL200800000004,26.740000,26.570000,7.690000,2.860000,100.000000,0.000000
4,4,AL200800000005,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095


In [13]:
# Droping the column "Unnamed: 0"
train_label1 = train_label.drop(columns = "Unnamed: 0")

In [14]:
# Merging it with df3 using DHSID
train_data = pd.merge(df3,train_label1,on = 'DHSID',how = 'inner')

In [15]:
train_data.head()

,DHSID,ET_water_median@CAS/IGSNRR/PML/V2&timestamped,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,qc_median@CAS/IGSNRR/PML/V2&timestamped,ET_water_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_mean@CAS/IGSNRR/PML/V2&timestamped,Ei_mean@CAS/IGSNRR/PML/V2&timestamped,...,motorized_only_travel_time_100000,key1,key2,key3,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000001,-0.802911,0.697994,0.051240,0.720611,2.867798,0.0,-0.385094,1.004067,0.063028,...,47.410877,AL,1,2008,24.120000,25.280000,50.000000,9.680000,100.000000,20.000000
1,AL200800000002,-0.802911,0.546368,0.072052,0.641110,2.831818,0.0,-0.385094,0.949528,0.080869,...,38.932705,AL,2,2008,23.040000,21.980000,7.690000,8.330000,66.670000,0.000000
2,AL200800000003,-0.802911,0.474569,0.040593,0.809169,2.093236,0.0,-0.385094,0.784751,0.055926,...,43.192800,AL,3,2008,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
3,AL200800000004,-0.802911,0.793127,0.056218,0.680800,3.167071,0.0,-0.385094,1.091989,0.066411,...,46.634487,AL,4,2008,26.740000,26.570000,7.690000,2.860000,100.000000,0.000000
4,AL200800000005,-0.802911,0.689465,0.072462,0.696633,2.979739,0.0,-0.385094,0.930896,0.080958,...,43.546497,AL,5,2008,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095


Droping the duplicate rows if any present

In [16]:
# Droping the duplicates of column DHSID
train_data.drop_duplicates(subset=[ "DHSID"], keep="first", inplace=True)

Finding any columns present with missing values

In [17]:
columns_with_missing_values = train_data.columns[train_data.isna().any()].tolist()

In [18]:
columns_with_missing_values

['Deep_Blue_Single_Scattering_Albedo_Land_Mean_Mean_412_median@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Std_Deviation_Mean_412_median@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Mean_Mean_412_mean@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Std_Deviation_Mean_412_mean@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Mean_Mean_412_max_max@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Mean_Mean_412_max_min@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Mean_Mean_412_min_max@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Mean_Mean_412_min_min@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Std_Deviation_Mean_412_max_max@MODIS/061/MOD08_M3&timestamped',
 'Deep_Blue_Single_Scattering_Albedo_Land_Std_Deviation_Mean_412_max_min@MODIS/061/MOD08_M3&timestamped',
 '

In [19]:
# Assuming these columns doesnot give that much importance, droping these columns also
train_data = train_data.drop(columns = columns_with_missing_values)

Droping the DHSID column

In [20]:
# Droping the DHSID column
train_data1 = train_data.drop(columns ="DHSID")

# Reseting the index
train_data1.reset_index(drop = True,inplace = True)
train_data1.head()

,ET_water_median@CAS/IGSNRR/PML/V2&timestamped,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,qc_median@CAS/IGSNRR/PML/V2&timestamped,ET_water_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_mean@CAS/IGSNRR/PML/V2&timestamped,Ei_mean@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,...,motorized_only_travel_time_100000,key1,key2,key3,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,-0.802911,0.697994,0.051240,0.720611,2.867798,0.0,-0.385094,1.004067,0.063028,0.703046,...,47.410877,AL,1,2008,24.120000,25.280000,50.000000,9.680000,100.000000,20.000000
1,-0.802911,0.546368,0.072052,0.641110,2.831818,0.0,-0.385094,0.949528,0.080869,0.636434,...,38.932705,AL,2,2008,23.040000,21.980000,7.690000,8.330000,66.670000,0.000000
2,-0.802911,0.474569,0.040593,0.809169,2.093236,0.0,-0.385094,0.784751,0.055926,0.764483,...,43.192800,AL,3,2008,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
3,-0.802911,0.793127,0.056218,0.680800,3.167071,0.0,-0.385094,1.091989,0.066411,0.676971,...,46.634487,AL,4,2008,26.740000,26.570000,7.690000,2.860000,100.000000,0.000000
4,-0.802911,0.689465,0.072462,0.696633,2.979739,0.0,-0.385094,0.930896,0.080958,0.634081,...,43.546497,AL,5,2008,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095


Label encoding for the columns dtype==object

In [21]:
from sklearn import preprocessing

# Looping over each column in the train_data1 where dtype is "object"
for column in train_data1.select_dtypes(include=["object"]).columns:
    
    # Initializing a Label Encoder object
    label_encoder = preprocessing.LabelEncoder()
    
    #  Fiting the encoder to the unique values in the column
    label_encoder.fit(train_data1[column].unique())
    
    # Transforming the column using the encoder
    train_data1[column] = label_encoder.transform(train_data1[column])
    
    # Printing the column name and the unique encoded values
    print(f"{column}: {train_data1[column].unique()}")  

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


key1: [0]


In [22]:
# Six health indicator column names we want to predict
column_names =  ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

Splitting data into X and Y for analysis

In [23]:
X = train_data1.drop(columns = column_names)  # X
Y = train_data1[column_names]                 # Y

Using **Random Forest Regressor** for training this data

In [24]:
from sklearn.ensemble import RandomForestRegressor

# Using RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, Y)

RandomForestRegressor(random_state=42)

Extracting the **top 2000 features**

In [25]:
# Extracting feature importances from the trained model
rf_feature_importances = model.feature_importances_

In [26]:
# Creating a DataFrame to store feature importances and their corresponding feature indices
feature_importance_df = pd.DataFrame({'Feature': X.columns,
                                      'Importance': rf_feature_importances})

In [27]:
# Feature importance dataframe
feature_importance_df

,Feature,Importance
0,ET_water_median@CAS/IGSNRR/PML/V2&timestamped,0.000005
1,Ec_median@CAS/IGSNRR/PML/V2&timestamped,0.000158
2,Ei_median@CAS/IGSNRR/PML/V2&timestamped,0.000153
3,Es_median@CAS/IGSNRR/PML/V2&timestamped,0.000485
4,GPP_median@CAS/IGSNRR/PML/V2&timestamped,0.000309
...,...,...
11316,motorized_only_friction_surface_100000,0.001187
11317,motorized_only_travel_time_100000,0.000370
11318,key1,0.000000
11319,key2,0.014468


In [28]:
# Sorting the DataFrame based on importance in descending order, to extract the top features
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)

In [29]:
# Select the top 1999 features
selected_features = feature_importance_df.head(1999)['Feature'].values.tolist()

In [30]:
# Also adding 'key1' as a features, since 'key1' gives an idea of the country names
selected_features.append('key1')

In [31]:
selected_features

['key3',
 'key2',
 'bio16@WORLDCLIM/V1/BIO',
 'sensor_zenith_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped',
 'sur_refl_b02_median@MODIS/006/MYD09A1&timestamped',
 'v_component_of_wind_10m_mean@ECMWF/ERA5_LAND/MONTHLY&timestamped',
 'Wind_f_tavg_median@NASA/FLDAS/NOAH01/C/GL/M/V001&timestamped',
 'u_component_of_wind_10m_max_max@ECMWF/ERA5/MONTHLY&timestamped',
 'susm_max_min@NASA_USDA/HSL/soil_moisture&timestamped',
 'u_component_of_wind_10m_max_min@ECMWF/ERA5/MONTHLY&timestamped',
 'total_precipitation_median@ECMWF/ERA5/MONTHLY&timestamped',
 'bio15@WORLDCLIM/V1/BIO',
 'pet_min_min@IDAHO_EPSCOR/TERRACLIMATE&timestamped',
 'bio16_max@WORLDCLIM/V1/BIO',
 'LST_Day_1km_max_min@MODIS/006/MYD11A2&timestamped',
 'Mean_min_min@Oxford/MAP/LST_Day_5km_Monthly&timestamped',
 'sur_refl_b01_min_min@MODIS/006/MYD13A1&timestamped',
 'SO2_column_number_density_amf_mean@COPERNICUS/S5P/NRTI/L3_SO2&timestamped',
 'BRDF_Albedo_Parameters_Band6_vol_min_max@MODIS/006/MCD43A1&timestamped',
 'Mean_mi

Given just above is the selected ***2000 features***, we are using only this features or whole analysis

In [32]:
# Also adding DHSID column, since we want to merge the other files using this DHSID columns
selected_features.append('DHSID')

In [33]:
len(selected_features)

2001

**Now reading the gee_features with these 2001 features for analysis**

In [34]:
df_gee = pd.read_csv('/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv',usecols = selected_features)

In [35]:
df_gee.head()

,DHSID,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,AL200800000001,0.697994,0.051240,0.720611,2.867798,0.703046,0.0,4.486412,0.058939,0.442405,...,226.56590,0.025174,52.417587,0.043573,216.05801,0.014725,47.410877,AL,1,2008
1,AL200800000002,0.546368,0.072052,0.641110,2.831818,0.636434,0.0,3.593682,0.046119,0.350706,...,225.83365,0.029437,56.164467,0.039097,200.14983,0.012322,38.932705,AL,2,2008
2,AL200800000003,0.474569,0.040593,0.809169,2.093236,0.764483,0.0,3.358742,0.031136,0.325511,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3,2008
3,AL200800000004,0.793127,0.056218,0.680800,3.167071,0.676971,0.0,3.753731,0.058352,0.367744,...,233.25563,0.025747,53.518450,0.043162,216.46187,0.014495,46.634487,AL,4,2008
4,AL200800000005,0.689465,0.072462,0.696633,2.979739,0.634081,0.0,3.353964,0.043956,0.346370,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5,2008


Merging df_gee with preprocessed training_label for getting training data

In [36]:
# Merging the df_gee with training_label1 using DHSID column
train_data = pd.merge(df_gee,train_label1,on = 'DHSID',how = 'inner')

In [37]:
# Droping the column DHSID
train_data.drop_duplicates(subset=[ "DHSID"], keep="first", inplace=True)

Checking any missing values present

In [38]:
columns_with_missing_values_1 = train_data.columns[train_data.isna().any()].tolist()

In [39]:
columns_with_missing_values_1

[]

This implies that no missing values are present

In [40]:
# Reading the submission file
submission = pd.read_csv("/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv")

In [41]:
# Merging the df_gee with submission file including all data from the submission file
submission1 = pd.merge(df_gee,submission,on ='DHSID',how ='right')

In [42]:
submission1.head()

,DHSID,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,...,motorized_only_travel_time_100000,key1,key2,key3,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,0.474569,0.040593,0.809169,2.093236,0.764483,0.0,3.358742,0.031136,0.325511,...,43.192800,AL,3.0,2008.0,20,20,50,50,50,50
1,AL200800000005,0.689465,0.072462,0.696633,2.979739,0.634081,0.0,3.353964,0.043956,0.346370,...,43.546497,AL,5.0,2008.0,20,20,50,50,50,50
2,AL200800000007,0.526604,0.057890,0.727580,2.453626,0.686621,0.0,3.173691,0.033165,0.300231,...,39.679590,AL,7.0,2008.0,20,20,50,50,50,50
3,AL200800000008,0.373835,0.025905,0.843817,1.469125,0.968380,0.0,3.587190,0.000000,0.576523,...,39.403683,AL,8.0,2008.0,20,20,50,50,50,50
4,AL200800000009,0.526604,0.039826,0.784812,2.079996,0.785890,0.0,3.088861,0.000000,0.287542,...,39.236156,AL,9.0,2008.0,20,20,50,50,50,50


In [43]:
# Droping the dulicate DHSIDs by keeping first one
submission1.drop_duplicates(subset=[ "DHSID"], keep="first", inplace=True)

In [44]:
# Column names that we want to predict
column_names =  ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [45]:
# Droping these columns that we want to predict
submission2 = submission1.drop(columns = column_names)

In [46]:
submission2.head()

,DHSID,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,AL200800000003,0.474569,0.040593,0.809169,2.093236,0.764483,0.0,3.358742,0.031136,0.325511,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3.0,2008.0
1,AL200800000005,0.689465,0.072462,0.696633,2.979739,0.634081,0.0,3.353964,0.043956,0.346370,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5.0,2008.0
2,AL200800000007,0.526604,0.057890,0.727580,2.453626,0.686621,0.0,3.173691,0.033165,0.300231,...,228.48878,0.028704,55.940290,0.039564,203.26802,0.012489,39.679590,AL,7.0,2008.0
3,AL200800000008,0.373835,0.025905,0.843817,1.469125,0.968380,0.0,3.587190,0.000000,0.576523,...,233.96854,0.028273,56.945385,0.040597,202.23082,0.012181,39.403683,AL,8.0,2008.0
4,AL200800000009,0.526604,0.039826,0.784812,2.079996,0.785890,0.0,3.088861,0.000000,0.287542,...,230.49530,0.028583,56.529280,0.040002,201.76999,0.012244,39.236156,AL,9.0,2008.0


In [47]:
# Missing columns of submission2 file
columns_with_missing_values_submission = submission2.columns[submission2.isna().any()].tolist()

In [48]:
len(columns_with_missing_values_submission)

2000

Except DHSID column, all other features have some missing values

In [49]:
# Finding the indices of the missing values rows
key2_missing_indices = []
for i, value in enumerate(submission2['key2']):
    if pd.isnull(value):
        key2_missing_indices.append(i)


In [50]:
key2_missing_indices

[3742,
 3756,
 3785,
 3791,
 3797,
 4530,
 4542,
 4550,
 4573,
 4581,
 4595,
 4596,
 4607,
 4608,
 4634,
 4665,
 4666,
 4719,
 4736,
 4744,
 4751,
 4757,
 4764,
 4777,
 4778]

Rows with these indices having whole data missing

In [51]:
# Storing the original data types of 'key1',key2' and 'key3'
original_key1_dtype = submission1['key1'].dtype
original_key2_dtype = submission1['key2'].dtype
original_key3_dtype = submission1['key3'].dtype

In [52]:
# Using pd.isna() to check for NaN values
if submission2['key2'].isna().any():
    # Selecting the rows in 'submission2' where 'key2' is NaN and the DHSID starts with 'DHS'
    missing_dhsids = submission2[submission2['key2'].isna() & submission2['DHSID'].str.startswith('DHS')]

    # Filling the missing 'key2' and 'key3' values based on the DHSID pattern
    submission2.loc[missing_dhsids.index, 'key3'] = missing_dhsids['DHSID'].str[3:7]
    submission2.loc[missing_dhsids.index, 'key2'] = missing_dhsids['DHSID'].str[7:]

In [54]:
missing_dhsids

,DHSID,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
3743,DHS20180000402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3757,DHS20180000452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3786,DHS20180000554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3792,DHS20180000579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3798,DHS20180000604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Using pd.isna() to check for NaN values
if submission2['key2'].isna().any():
    # Selecting the rows in 'submission2' where 'key2' is NaN and the DHSID does not start with 'DHS'
    missing_non_dhsids = submission2[submission2['key2'].isna() & ~submission2['DHSID'].str.startswith('DHS')]

    # Filling the missing 'key2' and 'key3' values based on the DHSID pattern
    submission2.loc[missing_non_dhsids.index, 'key3'] = missing_non_dhsids['DHSID'].str[2:6]
    submission2.loc[missing_non_dhsids.index, 'key2'] = missing_non_dhsids['DHSID'].str[6:]


In [55]:
missing_non_dhsids.iloc[:,0]

4531    EG201403230104
4543    EG201403480201
4551    EG201403610201
4574    EG201403980103
4582    EG201404100304
4596    EG201404340105
4597    EG201404350102
4608    EG201404452003
4609    EG201404460201
4635    EG201404950402
4666    EG201405500103
4667    EG201405500107
4720    EG201406490204
4737    EG201406740602
4745    EG201406870404
4752    EG201406992212
4758    EG201407110104
4765    EG201407240101
4778    EG201407480104
4779    EG201407490204
Name: DHSID, dtype: object

In [56]:
# Filling the 'key1' based on the forward fill method
submission2['key1'].fillna(method='ffill', inplace=True)

In [57]:
# Converting 'key1',key2' and 'key3' back to their original data types
submission2['key1'] = submission2['key1'].astype(original_key1_dtype)
submission2['key2'] = submission2['key2'].astype(original_key2_dtype)
submission2['key3'] = submission2['key3'].astype(original_key3_dtype)

Now checking any missing values remaining

In [58]:
# Finding the indices of the missing values rows after filling those values
key2_missing_indices_after_filling = []
for i, value in enumerate(submission2['key2']):
    if pd.isnull(value):
        key2_missing_indices_after_filling.append(i)

In [60]:
# Checking the indices
key2_missing_indices_after_filling

[]

This implies that no misssing '**key**' values present in the submission file

In [61]:
# Random checking
submission2[submission2["DHSID"]=='DHS20180000452']

,DHSID,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
3757,DHS20180000452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BD,452.0,2018.0


In [63]:
submission2.reset_index(drop = True,inplace = True)
submission2.head()

,DHSID,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,AL200800000003,0.474569,0.040593,0.809169,2.093236,0.764483,0.0,3.358742,0.031136,0.325511,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3.0,2008.0
1,AL200800000005,0.689465,0.072462,0.696633,2.979739,0.634081,0.0,3.353964,0.043956,0.346370,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5.0,2008.0
2,AL200800000007,0.526604,0.057890,0.727580,2.453626,0.686621,0.0,3.173691,0.033165,0.300231,...,228.48878,0.028704,55.940290,0.039564,203.26802,0.012489,39.679590,AL,7.0,2008.0
3,AL200800000008,0.373835,0.025905,0.843817,1.469125,0.968380,0.0,3.587190,0.000000,0.576523,...,233.96854,0.028273,56.945385,0.040597,202.23082,0.012181,39.403683,AL,8.0,2008.0
4,AL200800000009,0.526604,0.039826,0.784812,2.079996,0.785890,0.0,3.088861,0.000000,0.287542,...,230.49530,0.028583,56.529280,0.040002,201.76999,0.012244,39.236156,AL,9.0,2008.0


In [64]:
# Droping the column "DHSID"
submission3 = submission2.drop(columns = ["DHSID"])

In [65]:
# Reseting the index after droping
submission3.reset_index(drop = True,inplace = True)

In [66]:
submission3.head()

,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Ei_min_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,0.474569,0.040593,0.809169,2.093236,0.764483,0.0,3.358742,0.031136,0.325511,0.000020,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3.0,2008.0
1,0.689465,0.072462,0.696633,2.979739,0.634081,0.0,3.353964,0.043956,0.346370,0.000017,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5.0,2008.0
2,0.526604,0.057890,0.727580,2.453626,0.686621,0.0,3.173691,0.033165,0.300231,0.000016,...,228.48878,0.028704,55.940290,0.039564,203.26802,0.012489,39.679590,AL,7.0,2008.0
3,0.373835,0.025905,0.843817,1.469125,0.968380,0.0,3.587190,0.000000,0.576523,0.000017,...,233.96854,0.028273,56.945385,0.040597,202.23082,0.012181,39.403683,AL,8.0,2008.0
4,0.526604,0.039826,0.784812,2.079996,0.785890,0.0,3.088861,0.000000,0.287542,0.000016,...,230.49530,0.028583,56.529280,0.040002,201.76999,0.012244,39.236156,AL,9.0,2008.0


In [67]:
# Making a copy of the submission3 file
submission4 = submission3.copy()

In [68]:
# Finding the missing columns in the submission4 file
columns_with_missing_values_submission4 = submission4.columns[submission4.isna().any()].tolist()

In [69]:
len(columns_with_missing_values_submission4)

1997

Except 3 columns all have missing values

Imputing the **missing values** with the **mean** of the features, based on **country basis**

In [71]:
# Adding missing values with the mean of the data of countries
from sklearn.impute import SimpleImputer

# Grouping the values based on countries
grouped_country = submission4.groupby(["key1"])

# Iterating over each group in (country_codes,group values)
for country, group in grouped_country:
    
    # Iterating over each columns in column_names
    for column in columns_with_missing_values_submission4:
        
        # Skipping imputation, if all values of a column is Nan
        if group[column].isnull().all():
            continue
            
        # Finding the missing values using 'mean'
        else:
            imputer = SimpleImputer(strategy = 'mean')
            group[column] = imputer.fit_transform(group[[column]])
    
    # Filling missing values in the group directly
    group.fillna(group.mean(), inplace=True)
    
    # Updating the DataFrame submission4 with new values
    submission4.loc[group.index] = group

/tmp/ipykernel_33/3063345913.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for country, group in grouped_country:
/tmp/ipykernel_33/3063345913.py:23: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  group.fillna(group.mean(), inplace=True)
/tmp/ipykernel_33/3063345913.py:23: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  group.fillna(group.mean(), inplace=T

Checking any missing values remaining

In [72]:
# Finding the missing columns in the submission4 file
columns_with_missing_values_submission4_after_imputing = submission4.columns[submission4.isna().any()].tolist()
columns_with_missing_values_submission4_after_imputing

[]

This shows that we have dealt with all missing values

In [73]:
# Droping the column DHSID from train_data
train_data1 = train_data.drop(columns ="DHSID")

In [74]:
# Reseting the index
train_data1.reset_index(drop = True,inplace = True)
train_data1

,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Ei_min_max@CAS/IGSNRR/PML/V2&timestamped,...,motorized_only_travel_time_100000,key1,key2,key3,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,0.697994,0.051240,0.720611,2.867798,0.703046,0.000000,4.486412,0.058939,0.442405,0.000016,...,47.410877,AL,1,2008,24.120000,25.280000,50.000000,9.680000,100.000000,20.000000
1,0.546368,0.072052,0.641110,2.831818,0.636434,0.000000,3.593682,0.046119,0.350706,0.000019,...,38.932705,AL,2,2008,23.040000,21.980000,7.690000,8.330000,66.670000,0.000000
2,0.474569,0.040593,0.809169,2.093236,0.764483,0.000000,3.358742,0.031136,0.325511,0.000020,...,43.192800,AL,3,2008,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
3,0.793127,0.056218,0.680800,3.167071,0.676971,0.000000,3.753731,0.058352,0.367744,0.000014,...,46.634487,AL,4,2008,26.740000,26.570000,7.690000,2.860000,100.000000,0.000000
4,0.689465,0.072462,0.696633,2.979739,0.634081,0.000000,3.353964,0.043956,0.346370,0.000017,...,43.546497,AL,5,2008,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77943,0.988795,0.006372,0.622020,2.953615,0.867226,0.567900,3.114433,0.000000,0.885903,0.000000,...,36.612495,ZW,396,2015,23.160000,22.140000,33.330000,2.040000,78.950000,25.000000
77944,1.225136,0.021697,0.694095,4.129716,0.686939,0.328377,3.072238,0.281573,0.422164,0.000000,...,38.772083,ZW,397,2015,24.330000,22.610000,11.110000,1.750000,76.190000,22.220000
77945,1.629395,0.025855,0.551693,3.774360,0.431139,1.800580,3.441210,0.625408,0.969633,0.000000,...,65.126240,ZW,398,2015,23.700000,21.440000,10.530000,4.880000,68.000000,17.650000
77946,0.000000,0.000000,0.594644,0.000000,1.191415,0.886881,1.678322,0.000000,0.252395,0.000000,...,40.164307,ZW,399,2015,23.719336,22.992242,17.691171,6.484954,75.459806,30.248275


In [75]:
# Column names we want to predict
column_names =  ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [76]:
column_names

['Mean_BMI',
 'Median_BMI',
 'Unmet_Need_Rate',
 'Under5_Mortality_Rate',
 'Skilled_Birth_Attendant_Rate',
 'Stunted_Rate']

In [78]:
# y_train, ie the 'column_names' from the train_data1
y_train = train_data1[column_names]
y_train

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,24.120000,25.280000,50.000000,9.680000,100.000000,20.000000
1,23.040000,21.980000,7.690000,8.330000,66.670000,0.000000
2,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
3,26.740000,26.570000,7.690000,2.860000,100.000000,0.000000
4,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
...,...,...,...,...,...,...
77943,23.160000,22.140000,33.330000,2.040000,78.950000,25.000000
77944,24.330000,22.610000,11.110000,1.750000,76.190000,22.220000
77945,23.700000,21.440000,10.530000,4.880000,68.000000,17.650000
77946,23.719336,22.992242,17.691171,6.484954,75.459806,30.248275


In [79]:
# x_train, after droping 'column_names' from the train_data1
x_train = train_data1.drop(columns = column_names)
x_train

,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Ei_min_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,0.697994,0.051240,0.720611,2.867798,0.703046,0.000000,4.486412,0.058939,0.442405,0.000016,...,226.565900,0.025174,52.417587,0.043573,216.05801,0.014725,47.410877,AL,1,2008
1,0.546368,0.072052,0.641110,2.831818,0.636434,0.000000,3.593682,0.046119,0.350706,0.000019,...,225.833650,0.029437,56.164467,0.039097,200.14983,0.012322,38.932705,AL,2,2008
2,0.474569,0.040593,0.809169,2.093236,0.764483,0.000000,3.358742,0.031136,0.325511,0.000020,...,225.121170,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,AL,3,2008
3,0.793127,0.056218,0.680800,3.167071,0.676971,0.000000,3.753731,0.058352,0.367744,0.000014,...,233.255630,0.025747,53.518450,0.043162,216.46187,0.014495,46.634487,AL,4,2008
4,0.689465,0.072462,0.696633,2.979739,0.634081,0.000000,3.353964,0.043956,0.346370,0.000017,...,249.495010,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,AL,5,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77943,0.988795,0.006372,0.622020,2.953615,0.867226,0.567900,3.114433,0.000000,0.885903,0.000000,...,95.517525,0.010494,30.999570,0.014701,111.36423,0.011361,36.612495,ZW,396,2015
77944,1.225136,0.021697,0.694095,4.129716,0.686939,0.328377,3.072238,0.281573,0.422164,0.000000,...,78.550600,0.013001,37.956570,0.015073,81.61242,0.012609,38.772083,ZW,397,2015
77945,1.629395,0.025855,0.551693,3.774360,0.431139,1.800580,3.441210,0.625408,0.969633,0.000000,...,174.991070,0.012722,65.993355,0.014846,136.68535,0.013000,65.126240,ZW,398,2015
77946,0.000000,0.000000,0.594644,0.000000,1.191415,0.886881,1.678322,0.000000,0.252395,0.000000,...,97.999340,0.010908,33.700840,0.014639,126.11693,0.011399,40.164307,ZW,399,2015


In [80]:
# DHSID column of the submission file
DHSID = submission2["DHSID"]
DHSID

0        AL200800000003
1        AL200800000005
2        AL200800000007
3        AL200800000008
4        AL200800000009
              ...      
14995    ZW201500000382
14996    ZW201500000383
14997    ZW201500000386
14998    ZW201500000390
14999    ZW201500000399
Name: DHSID, Length: 15000, dtype: object

In [81]:
# Stacking the DataFrames vertically(ie, x_train and submission4)
merged_df = pd.concat([x_train, submission4], axis=0)

**Label Encoding**

In [82]:
from sklearn import preprocessing

# Separating the column names from the DataFrame
column_names = merged_df.columns.tolist()

In [83]:
# Looping over each column in the merged_df where dtype is "object"
for column in merged_df.select_dtypes(include=["object"]).columns:
    
    # Initializing a Label Encoder object
    label_encoder = preprocessing.LabelEncoder()
    
    #  Fiting the encoder to the uique values in the column
    label_encoder.fit(merged_df[column].unique())
    
    # Transforming the column using the encoder
    merged_df[column] = label_encoder.transform(merged_df[column])
    
    # Printing the column name and the unique encoded values
    print(f"{column}: {merged_df[column].unique()}")  

key1: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]


In [84]:
# Separating the DataFrame back into the original parts
x_train = merged_df[:len(x_train)]
x_submission = merged_df[len(x_train):]

Saving them into **csv files**, for **model training**

In [85]:
x_train.to_csv('x_train.csv',index = False)
y_train.to_csv('y_train.csv',index = False)
x_submission.to_csv('x_submission.csv',index = False)
DHSID.to_csv('DHSID.csv',index = False)

**x_train**

In [86]:
x_train

,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Ei_min_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,0.697994,0.051240,0.720611,2.867798,0.703046,0.000000,4.486412,0.058939,0.442405,0.000016,...,226.565900,0.025174,52.417587,0.043573,216.05801,0.014725,47.410877,0,1.0,2008.0
1,0.546368,0.072052,0.641110,2.831818,0.636434,0.000000,3.593682,0.046119,0.350706,0.000019,...,225.833650,0.029437,56.164467,0.039097,200.14983,0.012322,38.932705,0,2.0,2008.0
2,0.474569,0.040593,0.809169,2.093236,0.764483,0.000000,3.358742,0.031136,0.325511,0.000020,...,225.121170,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,0,3.0,2008.0
3,0.793127,0.056218,0.680800,3.167071,0.676971,0.000000,3.753731,0.058352,0.367744,0.000014,...,233.255630,0.025747,53.518450,0.043162,216.46187,0.014495,46.634487,0,4.0,2008.0
4,0.689465,0.072462,0.696633,2.979739,0.634081,0.000000,3.353964,0.043956,0.346370,0.000017,...,249.495010,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,0,5.0,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77943,0.988795,0.006372,0.622020,2.953615,0.867226,0.567900,3.114433,0.000000,0.885903,0.000000,...,95.517525,0.010494,30.999570,0.014701,111.36423,0.011361,36.612495,49,396.0,2015.0
77944,1.225136,0.021697,0.694095,4.129716,0.686939,0.328377,3.072238,0.281573,0.422164,0.000000,...,78.550600,0.013001,37.956570,0.015073,81.61242,0.012609,38.772083,49,397.0,2015.0
77945,1.629395,0.025855,0.551693,3.774360,0.431139,1.800580,3.441210,0.625408,0.969633,0.000000,...,174.991070,0.012722,65.993355,0.014846,136.68535,0.013000,65.126240,49,398.0,2015.0
77946,0.000000,0.000000,0.594644,0.000000,1.191415,0.886881,1.678322,0.000000,0.252395,0.000000,...,97.999340,0.010908,33.700840,0.014639,126.11693,0.011399,40.164307,49,399.0,2015.0


**y_train**

In [87]:
y_train

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,24.120000,25.280000,50.000000,9.680000,100.000000,20.000000
1,23.040000,21.980000,7.690000,8.330000,66.670000,0.000000
2,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
3,26.740000,26.570000,7.690000,2.860000,100.000000,0.000000
4,25.059406,24.434744,30.633723,1.753384,71.819921,16.425095
...,...,...,...,...,...,...
77943,23.160000,22.140000,33.330000,2.040000,78.950000,25.000000
77944,24.330000,22.610000,11.110000,1.750000,76.190000,22.220000
77945,23.700000,21.440000,10.530000,4.880000,68.000000,17.650000
77946,23.719336,22.992242,17.691171,6.484954,75.459806,30.248275


**x_submission**

In [88]:
x_submission

,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,Ec_min_min@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Ei_min_max@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,key1,key2,key3
0,0.474569,0.040593,0.809169,2.093236,0.764483,0.000000,3.358742,0.031136,0.325511,0.000020,...,225.12117,0.026694,52.411550,0.039717,214.10818,0.013806,43.192800,0,3.0,2008.0
1,0.689465,0.072462,0.696633,2.979739,0.634081,0.000000,3.353964,0.043956,0.346370,0.000017,...,249.49501,0.027163,58.282570,0.043852,206.45757,0.012748,43.546497,0,5.0,2008.0
2,0.526604,0.057890,0.727580,2.453626,0.686621,0.000000,3.173691,0.033165,0.300231,0.000016,...,228.48878,0.028704,55.940290,0.039564,203.26802,0.012489,39.679590,0,7.0,2008.0
3,0.373835,0.025905,0.843817,1.469125,0.968380,0.000000,3.587190,0.000000,0.576523,0.000017,...,233.96854,0.028273,56.945385,0.040597,202.23082,0.012181,39.403683,0,8.0,2008.0
4,0.526604,0.039826,0.784812,2.079996,0.785890,0.000000,3.088861,0.000000,0.287542,0.000016,...,230.49530,0.028583,56.529280,0.040002,201.76999,0.012244,39.236156,0,9.0,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.747184,0.014547,0.592953,2.632599,0.828747,0.388586,2.422143,0.270676,0.282589,0.000000,...,118.30832,0.012714,71.967224,0.014643,140.47174,0.012210,67.942795,49,382.0,2015.0
14996,0.942900,0.004192,0.570845,2.676295,0.591410,0.228348,3.471962,0.181286,0.414315,0.000000,...,115.19916,0.011598,40.954746,0.014095,143.71710,0.011881,50.976006,49,383.0,2015.0
14997,1.053559,0.000800,0.543463,3.001164,0.648765,0.228583,4.456654,0.000000,0.654535,0.000000,...,445.96393,0.010749,57.107490,0.016248,212.91324,0.009727,30.100767,49,386.0,2015.0
14998,0.000000,0.000000,0.601599,0.000000,1.288189,0.610581,3.670615,0.000000,1.014215,0.000000,...,97.46032,0.010383,31.199133,0.014669,120.61947,0.011312,38.552160,49,390.0,2015.0
